In [1]:
from collections import deque
class State:
    def __init__(self, board):
        self.board = board
        self.size = int(len(board) ** 0.5)
        self.blank_pos = board.index(0)
    
    def is_goal(self, goal):
        return self.board == goal
    
    def get_possible_moves(self):
        moves = []
        row, col = divmod(self.blank_pos, self.size)
        directions = {'UP': (-1, 0), 'DOWN': (1, 0), 'LEFT': (0, -1), 'RIGHT': (0, 1)}
        for action, (dr, dc) in directions.items():
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < self.size and 0 <= new_col < self.size:
                new_blank_pos = new_row * self.size + new_col
                new_board = self.board[:]
                new_board[self.blank_pos], new_board[new_blank_pos] = new_board[new_blank_pos], new_board[self.blank_pos]
                moves.append((action, State(new_board)))
        return moves


In [2]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
    
    def expand(self):
        return [Node(state, self, action, self.path_cost + 1) for action, state in self.state.get_possible_moves()]
    
    def solution(self):
        node, path = self, []
        while node:
            path.append(node.action)
            node = node.parent
        return path[::-1]
    
    def __lt__(self, other):
        return self.path_cost < other.path_cost


In [3]:
class Problem:
    def __init__(self, initial, goal):
        self.initial = initial
        self.goal = goal
    
    def goal_test(self, state):
        return state.is_goal(self.goal)


In [4]:
def bfs(problem):
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node.solution(), 0
    
    frontier = deque([node])
    explored = set()
    explored.add(tuple(node.state.board))
    nodes_expanded = 0
    
    while frontier:
        node = frontier.popleft()
        
        for child in node.expand():
            if tuple(child.state.board) not in explored:
                if problem.goal_test(child.state):
                    return child.solution(), nodes_expanded
                frontier.append(child)
                explored.add(tuple(child.state.board))
                nodes_expanded += 1
    return None, nodes_expanded


In [5]:
def dfs(problem):
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node.solution(), 0
    
    frontier = [node]
    explored = set()
    explored.add(tuple(node.state.board))
    nodes_expanded = 0
    
    while frontier:
        node = frontier.pop()
        
        for child in node.expand():
            if tuple(child.state.board) not in explored:
                if problem.goal_test(child.state):
                    return child.solution(), nodes_expanded
                frontier.append(child)
                explored.add(tuple(child.state.board))
                nodes_expanded += 1
    return None, nodes_expanded


In [6]:
def dls(node, problem, limit, explored):
    if problem.goal_test(node.state):
        return node.solution(), 0
    
    elif limit == 0:
        return 'cutoff', 0
    
    else:
        cutoff_occurred = False
        nodes_expanded = 0
        explored.add(tuple(node.state.board))
        for child in node.expand():
            if tuple(child.state.board) not in explored:
                result, nodes = dls(child, problem, limit - 1, explored)
                nodes_expanded += nodes
                if result == 'cutoff':
                    cutoff_occurred = True
                elif result is not None:
                    return result, nodes_expanded
        explored.remove(tuple(node.state.board))
        return ('cutoff' if cutoff_occurred else None), nodes_expanded


In [7]:
def ids(problem):
    depth = 0
    while True:
        node = Node(problem.initial)
        explored = set()
        result, nodes_expanded = dls(node, problem, depth, explored)
        if result != 'cutoff':
            return result, nodes_expanded
        depth += 1


In [8]:
initial_state = State([7, 1, 8, 0, 4, 6, 2, 3, 5])
goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
problem = Problem(initial_state, goal_state)

# Run BFS
bfs_solution, bfs_nodes_expanded = bfs(problem)
bfs_path_cost = len(bfs_solution) - 1

# Run DFS
dfs_solution, dfs_nodes_expanded = dfs(problem)
dfs_path_cost = len(dfs_solution) - 1

# Run IDS
ids_solution, ids_nodes_expanded = ids(problem)
ids_path_cost = len(ids_solution) - 1

# Print Results
print(f"BFS - Nodes Expanded: {bfs_nodes_expanded}, Path Cost: {bfs_path_cost}")
print(f"DFS - Nodes Expanded: {dfs_nodes_expanded}, Path Cost: {dfs_path_cost}")
print(f"IDS - Nodes Expanded: {ids_nodes_expanded}, Path Cost: {ids_path_cost}")


BFS - Nodes Expanded: 152482, Path Cost: 25
DFS - Nodes Expanded: 155347, Path Cost: 62165
IDS - Nodes Expanded: 0, Path Cost: 25


In [9]:
#BFS - Nodes Expanded: 152482, Path Cost: 25
#DFS - Nodes Expanded: 155347, Path Cost: 62165
#IDS - Nodes Expanded: 0, Path Cost: 25
